In [1]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from itertools import takewhile, chain
from time import sleep
import re
from bs4.element import Tag
from typing import List
from tqdm import tqdm
import requests
import json


In [4]:
data=pd.read_parquet('demo_links.parquet')
data=data.sort_values(by='date',ascending=False)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'demo_links.parquet'

In [5]:
#Iterates from label to label and gets text in between(don't ignore case)
#by taking the `html` variable from the question.
def extract_item_text(link,driver):
    dict={}
    # html=requests.get(link)
    driver.get(link)
    soup = BeautifulSoup(driver.page_source,'lxml')
    #This snipet accounts for change in case between 10ks
    tags=[]
    tags_og=soup.find_all(string=re.compile("ITEM"))
    tags_og+=soup.find_all(string=re.compile("Item"))
    for tag in tags_og:
        #Convert navigable string to string for comparison
        string = str(tag.string)
        if  string[0:4]=='Item' or string[0:4]=='ITEM':
            tags.append(tag)
    #Loop through all Item tags picked up
    # print(tags)
    for i in range(len(tags)): 
        p='' 
        #Gets all remaining text
        string=tags[i].find_all_next(string=True)
        #We loop through all remaining text and then cut it off when we see the next tag
        for w in range(len(string)):
            p+=string[w]
            if i+1>=len(tags):
                while w<len(string):
                    p+=string[w]
                    w+=1
                break
            if tags[i+1] in string[w]:
                break
        #Add cut off text to dictionary
        dict[tags[i]]=p
        # data.append(dict)
    return dict

In [5]:
with open('10ks.json', encoding='utf-8') as F:
    dict = json.loads(F.read())

In [23]:
import json
chrome_options=Options()
# chrome_options.add_argument('--headless')
dict={}
#Add path of chromedriver and add options
driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver',options=chrome_options) 
#links=data['link'].reset_index()
#links=links['link']
links = ['https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm']
i=0
for item in tqdm(links):
    if item not in dict.keys():
        # if i%10==0:
        #     json_file=json.dumps(dict)
        #     with open("10ks.json", "w") as outfile:
        #         outfile.write(json_file)
        result=extract_item_text(item,driver)
        dict[item]=result.copy()
        
        json_file=json.dumps(dict)
        with open("10ks.json", "w") as outfile:
            outfile.write(json_file)
        i+=1
#Change chromedriver to requests library to avoid error

/var/folders/m6/h9jtrhyn259fytxp6j9xh9v00000gn/T/ipykernel_17926/3436935186.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver',options=chrome_options)
100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [ ]:
dict

In [7]:
import json
json_file=json.dumps(dict)

In [8]:
with open("10ks.json", "w") as outfile:
    outfile.write(json_file)

In [ ]:
dict[links[956]]

In [25]:
from sec_api import ExtractorApi

In [26]:
extractorApi = ExtractorApi("399f1f12e5b8f64d78f9ac9c5fca3c1797221994f6137dac3549ba502556abf9")


In [4]:
fillings = pd.read_csv("10k_links.csv")

In [26]:
url = fillings.iloc[1][1]

In [27]:
sections_10k = ["1", "1A", "1B", "2", "3", "4", "5", "6", "7", "7A", "8", "9", "9A", "9B", "10", "11", "12", "13", "14"]

In [28]:
url

'https://www.sec.gov/Archives/edgar/data/1551152/000155115222000007/abbv-20211231.htm'

In [29]:
#soup = BeautifulSoup(requests.get(url).content, "html.parser")

In [30]:

driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver') 

/var/folders/m6/h9jtrhyn259fytxp6j9xh9v00000gn/T/ipykernel_23083/3920258546.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver')


In [36]:
driver.get(url)

html_source = driver.page_source

driver.close()

In [37]:
soup = BeautifulSoup(html_source)

In [13]:
def item_span(tag):
    print(tag.parent.contents[0])
    return tag.name=='span' and 'Item' in tag.parent.contents[0]

In [ ]:
tags = soup.findAll("span")

In [88]:
tags = soup.findAll("span")

In [38]:
temp = soup.find_all('span', text = re.compile('Item'))

In [39]:
temp = [ x for x in temp if "/a" not in str(x)]

In [40]:
temp

[<span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 1.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 1A.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 1B.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 2.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 3.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 4.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 5.</span>,
 <span style="color:#000000;font-family:'Calibri',sans-serif;font-size:10pt;font-weight:400;line-height:100%">Item 6

In [25]:
temp

[<span style="color:#000000;font-family:'Helvetica',sans-serif;font-size:9pt;font-style:italic;font-weight:400;line-height:120%">This Annual Report on Form 10-K (“Form 10-K”) contains forward-looking statements, within the meaning of the Private Securities Litigation Reform Act of 1995, that involve risks and uncertainties. Many of the forward-looking statements are located in Part I, Item 1 of this Form 10-K under the heading “Business” and Part II, Item 7 of this Form 10-K under the heading “Management’s Discussion and Analysis of Financial Condition and Results of Operations.” Forward-looking statements provide current expectations of future events based on certain assumptions and include any statement that does not directly relate to any historical or current fact. For example, statements in this Form 10-K regarding the potential future impact of the COVID-19 pandemic on the Company’s business and results of operations are forward-looking statements. Forward-looking statements can 

In [38]:


for section in sections_10k:
    section_header = "Item " + section
    

Item 1
Item 1A
Item 1B
Item 2
Item 3
Item 4
Item 5
Item 6
Item 7
Item 7A
Item 8
Item 9
Item 9A
Item 9B
Item 10
Item 11
Item 12
Item 13
Item 14


In [33]:
section_text = extractorApi.get_section(test, "1A", "text")

In [ ]:
section_text